## Optimized_Parser_approach CICIDS


This approach comprises of extracting pcap file features utilizing own developed pcap parser. Then utilizing it for comparison and labelling. Prominent steps for this approach are:
1. Extract Pcap file features into csv using developed parser. 
2. Perform labelling by comparing pre-processed UNSW CSV file and pcap parsed file (in .csv format).This approach compares both file using 5 distinct features which are :

> a)  Source IP Address(*srcip*) 
>
> b)  Destination IP Address(*dstip*)
>
> c)  Destination Port Number(*dsport*)
>
> d)  Source Port Numbe(*sport*)
>
> e)  Transaction Protocol(*protocol_m*)

3. Moreover, time duration is utilized to cross validate the attacks. The time duration is available at the official site of the dataset.
            



In [ ]:
import sys
sys.path.append("C:\\Users\\yasir.ali\\Project\\Payload-Byte")
## append path for your project

In [ ]:
import hashlib as hl
import csv
from datetime import datetime
from os import close
import numpy as np
import logging
import argparse
import sys
from datetime import datetime
from datetime import timezone
import socket
import math
import pickle
import random
import pandas as pd
from Functions.Functions_wireshark import *
from Functions.Optimized_Parser_Labelling import *
log = logging.getLogger()
log.setLevel(logging.DEBUG)

In [ ]:
## Checking ambiguity in the given time
df_p['Check']=df_p['Timestamp'].apply(lambda x: 100 if x.count(':') == 1 else 200)
df_p.Check.value_counts()

## Reading pre-processed CSV File of CICIDS


In [ ]:
df_p=pd.read_csv('F:/Project_Data/NIDS/CSV-Files-CICIDS-2017/Preprocessed-Data/CICIDS.csv')
df_p=df_p[['Timestamp','Source IP','Destination IP','Destination Port','Source Port','Protocol','Label']]
df_p.rename(columns={'Timestamp': 'stime', 'Source IP': 'srcip', 'Destination IP': 'dstip', 'Destination Port': 'dsport', 'Source Port': 'sport', 'Label': 'label','Protocol': 'protocol_m'}, inplace=True)

#Converting time into epoch format along with the mitigation of AM/PM issue.
df_p['stime']=df_p['stime'].apply(lambda x: (datetime.strptime(x, '%d/%m/%Y %H:%M')) if x.count(':') == 1 else  (datetime.strptime(x, '%d/%m/%Y %H:%M:%S')))
df_p['stime']=df_p['stime'].apply(lambda x: int((datetime(x.year,x.month,x.day,(x.hour+15),x.minute,x.second,tzinfo=timezone.utc)).timestamp()) if (x.hour>=1)&(x.hour<=7) else int((datetime(x.year,x.month,x.day,(x.hour+3),x.minute,x.second,tzinfo=timezone.utc)).timestamp()))
df_p=df_p.sort_values(by='stime')

In [ ]:
df_p['protocol_m'] = df_p['protocol_m'].astype(str)
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('6.0', 'tcp'))
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('17.0', 'udp'))
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('0.0', 'other'))

In [ ]:
df_p.label.value_counts()

## Parsing PCAP files into CSV

In [ ]:
pcap='F:/Project_Data/NIDS/Pcap-Files-CICIDS-2017/'
pcap_file=[]
days=['Monday','Tuesday','Wednesday','Thursday','Friday']
for x in days:
    pcap_file.append(pcap+x+"-WorkingHours"+".pcap")
    
    
out_file='F:/Project_Data/NIDS/CICIDS_result/pcap_file_csv_parser/'

In [ ]:
pcap_parser(pcap_file,out_file,1)

## Labeling & Combining of Parsed PCAP files

In [ ]:
df=pd.read_csv("F:/Project_Data/NIDS/CICIDS_result/pcap_file_csv_parser/pcap_csv_1.csv",index_col=0)
df=df.sort_values(by='stime')

In [ ]:
# df=df.sort_values(by='stime')
stime=df.stime[0]
ltime=int(df.stime.tail(1))
print("Start Value:",df.stime[0])
print("End Value:", int(df.stime.tail(1)))

In [ ]:
df.drop(columns=['frame_num','stime','ltime','protocol_s'],inplace=True)

In [ ]:
aa=df_p[(df_p['stime']>=stime) & (df_p['stime']<=ltime)]

In [ ]:
df_p.shape

In [ ]:
df.protocol_m.value_counts()

In [ ]:
aa.protocol_m.value_counts()

### `Labelling & Analyzing`


In [ ]:
combine=df.merge(aa, left_on=['srcip','dstip','dsport','sport','protocol_m'], right_on=['srcip','dstip','dsport','sport','protocol_m'])
combine

In [ ]:
combine.drop_duplicates(inplace=True)

In [ ]:
combine.shape

In [ ]:
combine.to_csv("F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/labelled_pcap_csv_1.csv",index=False)

## Cross Validation of Time Frame of Attacks

## Friday

#### Using time after transforming ADT time into UTC, since CSV file data is in ADT timezone and PCAP file data is in UTC+0:00. Therefore, to avoid any ambiguity transforming both of them to UTC.

In [ ]:
##  DDoS LOIT (15:56 – 16:16)
combine.drop(combine[(combine.stime>=1499453760 )&(combine.stime<=1499454960 )&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## Port Scan (13:55-15:30)
combine.drop(combine[(combine.stime>=1499446500)&(combine.stime<=1499452200)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## Botnet ARES (10:02 a.m. – 11:02 a.m.)
combine.drop(combine[(combine.stime>=1499432520)&(combine.stime<=1499436120)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
combine.shape

## Thursday

In [ ]:
## Infiltration – Dropbox download (15:04 – 15:45 p.m.)
combine.drop(combine[(combine.stime>=1499364240)&(combine.stime<=1499366700)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## Infiltration – Dropbox download and (14:33 -14:35)
combine.drop(combine[(combine.stime>=1499362380)&(combine.stime<=1499362500)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## Web Attack – Sql Injection (10:40 – 10:42 a.m.)
combine.drop(combine[(combine.stime>=1499348400)&(combine.stime<=1499348520)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## Web Attack – XSS (10:15 – 10:35 a.m.)
combine.drop(combine[(combine.stime>=1499346900)&(combine.stime<=1499348100)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## Web Attack – Brute Force (9:20 – 10 a.m.)
combine.drop(combine[(combine.stime>=1499343600)&(combine.stime<=1499346000)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
combine.shape

## Wednesday

In [ ]:
## Heartbleed Port 444 (15:12 - 15:32)
combine.drop(combine[(combine.stime>=1499278320)&(combine.stime<=1499279520)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## DoS GoldenEye (11:10 – 11:23 a.m.)
combine.drop(combine[(combine.stime>=1499263800)&(combine.stime<=1499264580)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## DoS Hulk (10:43 – 11 a.m.)
combine.drop(combine[(combine.stime>=1499262180)&(combine.stime<=1499263200)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
##DoS Slowhttptest (10:14 – 10:35 a.m.)
combine.drop(combine[(combine.stime>=1499260440)&(combine.stime<=1499261700)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
## DoS slowloris (9:47 – 10:10 a.m.)
combine.drop(combine[(combine.stime>=1499258820)&(combine.stime<=1499260200)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
combine.shape

## Tuesday

In [ ]:
## FTP-Patator (9:20 – 10:20 a.m.) 
combine.drop(combine[(combine.stime>=1499170800)&(combine.stime<=1499174400)&(combine.label=='BENIGN')].index,inplace=True)

In [ ]:
##SSH-Patator (14:00 – 15:00 p.m.)
combine.drop(combine[(combine.stime>=1499187600)&(combine.stime<=1499191200)&(combine.label=='BENIGN')].index,inplace=True)

## Reading and Combining All Labelled Files

In [ ]:
def combine_CICIDS(in_file_path,out_path):
    combine=pd.DataFrame(columns=['srcip', 'sport', 'dstip', 'dsport', 'protocol_m',
     'sttl', 'total_len', 'payload', 't_delta', 'stime','label'])
    for files in in_file_path:
        df=pd.read_csv(files)
        combine=combine.append(df, ignore_index=True)
        print(combine.shape)
    csv_out=out_path+"combined_labelled_pcap_csv.csv"
    logging.info("Exporting_combined_csv_file....")
    combine.to_csv(csv_out,index=False)
    return combine

In [ ]:
label_csv="F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/"
in_file=[]
for x in range(1,6): ## Starting & Ending values for the files
    in_file.append(label_csv+"labelled_pcap_csv_"+str(x)+".csv")
out_path="F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/Combined/"

In [ ]:
df_payload=combine_CICIDS(in_file,out_path)

In [ ]:
df_payload.shape

In [ ]:
df_payload.label.value_counts()

In [ ]:
df_payload.drop(df_payload[df_payload.payload.isnull()].index,inplace=True)

In [ ]:
df_payload.shape

In [ ]:
x=df_payload['payload']
new=[]
for p in range(len(x)):
    try:
        o=(int((x.iloc[p]), 16))
        if o>0:
            new.append(1)
        else:
            new.append(0)
    except Exception as e:
        print(p)
        new.append('Big')

In [ ]:
df_payload['payload_int']=new

In [ ]:
df_payload.drop(df_payload[df_payload.payload_int==0].index,inplace=True)

In [ ]:
## After Removing zero payload
df_payload.label.value_counts()

In [ ]:
df_payload.pop('payload_int')

In [ ]:
df_payload.sttl=df_payload.sttl.astype('int32')
df_payload.dsport=df_payload.dsport.astype('int32')
df_payload.sport=df_payload.sport.astype('int32')
df_payload.total_len=df_payload.total_len.astype('int32')

In [ ]:
df_payload.to_csv("F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/Combined/combined_labelled_cleaned_sorted_pcap_csv.csv",index=False)

In [ ]:
df_payload.iloc[:,:-1]

## Reducing Number of Instances for Dataset

In [ ]:
from imblearn.under_sampling import RandomUnderSampler 

In [ ]:
dict1={ 'BENIGN': 362108,
'DoS Hulk':          250000,
'DDoS'  :         241405,
'DoS GoldenEye':     128122,
'DoS slowloris':    121097,
'Infiltration'        :115007,
'DoS Slowhttptest'         :  80542,
'SSH-Patator':          48165,
'FTP-Patator'            :   31843,
'Heartbleed'              :  13486,
'Web Attack – Brute Force'            :   11754,
'Web Attack – XSS'              :  3341,
'Bot'            :   2543,
'PortScan'              :  830,
'Web Attack – Sql Injection': 12
}

In [ ]:
rus = RandomUnderSampler(random_state=42,sampling_strategy=dict1)

In [ ]:
X_res, y_res = rus.fit_resample(df_payload.iloc[:,:-1], df_payload.iloc[:,-1])

In [ ]:
X_res['label']=y_res

In [ ]:
X_res.to_csv("F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/Combined/combined_labelled_cleaned_undersampled_pcap_csv.csv",index=False)

## Transformation into Byte-wise features

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(X_res['protocol_m'])
X_res['protocol']=label

X_res.drop("protocol_m", axis=1, inplace=True)
X_res.drop("protocol_s", axis=1, inplace=True)
X_res.drop("frame_num", axis=1, inplace=True)

X_res['attack_cat']=y_res

In [ ]:
X_tr,Ytrain =payload_to_bytes(X_res,1500)

In [ ]:
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,6])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,7])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,5])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,9])))

In [ ]:
name=[]
for x in range(1,1501):
    name.append("payload_byte_"+str(x))
name.append("ttl")
name.append("total_len")
name.append("protocol")
name.append("t_delta")

In [ ]:
df = pd.DataFrame(X_tr, columns=name)
df['label']=Ytrain

In [ ]:
df.to_csv("F:/Converted_data_CICIDS.csv",index=False)

## Extra Code/ Conversion Epoch

In [ ]:
df_payload['stime'].fillna(df_payload['stime_y'],inplace=True)
df_payload.pop('stime_y')

### Transformation into epoch Format

In [ ]:
x='04/07/2017 09:20:00' 
x=(datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))
x=datetime(x.year,x.month,x.day,(x.hour+3),x.minute,x.second,tzinfo=timezone.utc)
x.timestamp()

In [ ]:
## AFter utc
int((datetime(x.year,x.month,x.day,(x.hour),x.minute,x.second,tzinfo=timezone.utc)).timestamp())